## Task 1: Python. Data Manipulation and usage of external APIs

In your first task, we would like you to consolidate your orders along with an exchange rate for the date of the order creations. This way, orders prices can be unified in Canadian Dollars (CAD), the currency that the Finance department of Namaste uses to report on.

For this purpose, we suggest you to use the free currency exchange rate API provided by: https://exchangeratesapi.io/

Your task is to implement a Python script that loads the orders, loads the exchange rate on the date of the order (via the Exchange Rate API) so each order contains a "currency rate" property from USD to CAD.

In [1]:
# Import required libraries

import pandas as pd
import requests
import json

In [2]:
# Import the data provided for the task
data = pd.read_json('orders.json')
data.head()

,id,customer,total_price,created_at,line_items
0,8369263756632563,"{'id': 47178, 'name': 'William Doe', 'email': ...",79.39,2020-03-07 14:31:11+00:00,"[{'id': 610448, 'product_id': 632910392, 'prod..."
1,7262170348080494,"{'id': 94720, 'name': 'Emile Tumson', 'email':...",86.98,2020-03-01 09:16:30+00:00,"[{'id': 997208, 'product_id': 278266679, 'prod..."
2,7722055557038194,"{'id': 59933, 'name': 'Ethan Jones', 'email': ...",36.99,2020-01-01 09:17:03+00:00,"[{'id': 759658, 'product_id': 573794972, 'prod..."
3,4280852170163518,"{'id': 23955, 'name': 'Daniel Smith', 'email':...",55.98,2020-03-10 11:59:46+00:00,"[{'id': 317808, 'product_id': 477365777, 'prod..."
4,5058732129957127,"{'id': 59933, 'name': 'Ethan Jones', 'email': ...",309.98,2020-03-15 07:27:24+00:00,"[{'id': 604665, 'product_id': 510650809, 'prod..."


#### Data Cleaning and Manipultion Steps

In [3]:
# Rename the column id to Order_id
data = data.rename(columns = {'id':'Order_id' })

In [4]:
# Convert the dictionaries of customer into separate columns
data = pd.concat([data.drop(['customer'], axis=1), data['customer'].apply(pd.Series)], axis=1)
data.head()

,Order_id,total_price,created_at,line_items,id,name,email
0,8369263756632563,79.39,2020-03-07 14:31:11+00:00,"[{'id': 610448, 'product_id': 632910392, 'prod...",47178,William Doe,william.doe@gmail.com
1,7262170348080494,86.98,2020-03-01 09:16:30+00:00,"[{'id': 997208, 'product_id': 278266679, 'prod...",94720,Emile Tumson,emile.tumson@gmail.com
2,7722055557038194,36.99,2020-01-01 09:17:03+00:00,"[{'id': 759658, 'product_id': 573794972, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com
3,4280852170163518,55.98,2020-03-10 11:59:46+00:00,"[{'id': 317808, 'product_id': 477365777, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com
4,5058732129957127,309.98,2020-03-15 07:27:24+00:00,"[{'id': 604665, 'product_id': 510650809, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com


In [5]:
# Rename the newly created columns
data = data.rename(columns = {'id':'customer_id','name':'customer_name' })

In [6]:
# Convert the dictionaries of line_items into separate columns
data = pd.concat([data.drop(['line_items'], axis=1), data['line_items'].apply(pd.Series)], axis=1)
data = data.drop([1, 2,3], axis=1)
data = data.rename(columns = {0:'line_items'})
data = pd.concat([data.drop(['line_items'], axis=1), data['line_items'].apply(pd.Series)], axis=1)
data = data.rename(columns = {'price':'product_price'})
data.head()

,Order_id,total_price,created_at,customer_id,customer_name,email,id,product_id,product_sku,product_name,product_price
0,8369263756632563,79.39,2020-03-07 14:31:11+00:00,47178,William Doe,william.doe@gmail.com,610448,632910392,PPLEPUNCH20-05,Purple Punch 2.0 Dried Flower,29.40
1,7262170348080494,86.98,2020-03-01 09:16:30+00:00,94720,Emile Tumson,emile.tumson@gmail.com,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99
2,7722055557038194,36.99,2020-01-01 09:17:03+00:00,59933,Ethan Jones,ethan.jones@gmail.com,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99
3,4280852170163518,55.98,2020-03-10 11:59:46+00:00,23955,Daniel Smith,daniel.smith@gmail.com,317808,477365777,GOGPR-07,Glueberry OG Pre-Rolls,5.99
4,5058732129957127,309.98,2020-03-15 07:27:24+00:00,59933,Ethan Jones,ethan.jones@gmail.com,604665,510650809,PAX-3,Pax 3 Vaporizer,259.99


####  free currency exchange rate API provided by: https://exchangeratesapi.io/

In [7]:
# Get the data from URL
r = requests.get(' https://api.exchangeratesapi.io/history?start_at=2020-01-01&end_at=2020-04-01&base=USD&symbols=CAD')
x = r.json()
df = pd.read_json(json.dumps(x))
df.head()

,rates,start_at,base,end_at
2020-01-02,{'CAD': 1.299830251},2020-01-01,USD,2020-04-01
2020-01-03,{'CAD': 1.2981968243},2020-01-01,USD,2020-04-01
2020-01-06,{'CAD': 1.2975701269},2020-01-01,USD,2020-04-01
2020-01-07,{'CAD': 1.2996777658},2020-01-01,USD,2020-04-01
2020-01-08,{'CAD': 1.3018443545},2020-01-01,USD,2020-04-01


In [8]:
# Deleting the non required columns
df = df.drop(df.columns[[1,2,3]], axis=1) 
df.head()

,rates
2020-01-02,{'CAD': 1.299830251}
2020-01-03,{'CAD': 1.2981968243}
2020-01-06,{'CAD': 1.2975701269}
2020-01-07,{'CAD': 1.2996777658}
2020-01-08,{'CAD': 1.3018443545}


In [9]:
# Assigning the index as column and rename the column
df.reset_index(level=0, inplace=True)
df = df.rename(columns = {'index':'date'})

In [10]:
## Convert the rate column into general columns and rename it
df = pd.concat([df.drop(['rates'], axis=1), df['rates'].apply(pd.Series)], axis=1)
df = df.rename(columns = {'CAD':'CAD_rate'})
df.head()

,date,CAD_rate
0,2020-01-02,1.299830
1,2020-01-03,1.298197
2,2020-01-06,1.297570
3,2020-01-07,1.299678
4,2020-01-08,1.301844


#### Few conversions in the main database  - 'created at' column type changed to standard type so that based on date currency excange rate can be moved to the mail file

In [11]:
data['date'] = data['created_at']
data['date'] = data['date'].dt.date
data.head()

,Order_id,total_price,created_at,customer_id,customer_name,email,id,product_id,product_sku,product_name,product_price,date
0,8369263756632563,79.39,2020-03-07 14:31:11+00:00,47178,William Doe,william.doe@gmail.com,610448,632910392,PPLEPUNCH20-05,Purple Punch 2.0 Dried Flower,29.40,2020-03-07
1,7262170348080494,86.98,2020-03-01 09:16:30+00:00,94720,Emile Tumson,emile.tumson@gmail.com,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,2020-03-01
2,7722055557038194,36.99,2020-01-01 09:17:03+00:00,59933,Ethan Jones,ethan.jones@gmail.com,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,2020-01-01
3,4280852170163518,55.98,2020-03-10 11:59:46+00:00,23955,Daniel Smith,daniel.smith@gmail.com,317808,477365777,GOGPR-07,Glueberry OG Pre-Rolls,5.99,2020-03-10
4,5058732129957127,309.98,2020-03-15 07:27:24+00:00,59933,Ethan Jones,ethan.jones@gmail.com,604665,510650809,PAX-3,Pax 3 Vaporizer,259.99,2020-03-15


In [12]:
# format the date column in the other file
df['date'] = df['date'].dt.date

In [13]:
# merging the files to get the CAD rate in the main file
data = pd.merge(data,df,on = 'date')
data.head()

,Order_id,total_price,created_at,customer_id,customer_name,email,id,product_id,product_sku,product_name,product_price,date,CAD_rate
0,4280852170163518,55.98,2020-03-10 11:59:46+00:00,23955,Daniel Smith,daniel.smith@gmail.com,317808,477365777,GOGPR-07,Glueberry OG Pre-Rolls,5.99,2020-03-10,1.367779
1,6431399815998774,315.97,2020-03-12 07:16:07+00:00,23955,Daniel Smith,daniel.smith@gmail.com,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,2020-03-12,1.381139
2,5307192536865304,86.98,2020-03-12 01:40:57+00:00,47178,William Doe,william.doe@gmail.com,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,2020-03-12,1.381139
3,5791957346866373,36.99,2020-03-24 09:52:40+00:00,47178,William Doe,william.doe@gmail.com,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,2020-03-24,1.449414
4,5235927587417736,36.99,2020-03-24 07:12:33+00:00,94720,Emile Tumson,emile.tumson@gmail.com,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,2020-03-24,1.449414


In [14]:
# convert the original order price and the product price to CAD
data['order_price_CAD'] = data['total_price']*data['CAD_rate']
data['product_price_CAD'] = data['product_price']*data['CAD_rate']
data.head()

,Order_id,total_price,created_at,customer_id,customer_name,email,id,product_id,product_sku,product_name,product_price,date,CAD_rate,order_price_CAD,product_price_CAD
0,4280852170163518,55.98,2020-03-10 11:59:46+00:00,23955,Daniel Smith,daniel.smith@gmail.com,317808,477365777,GOGPR-07,Glueberry OG Pre-Rolls,5.99,2020-03-10,1.367779,76.568255,8.192995
1,6431399815998774,315.97,2020-03-12 07:16:07+00:00,23955,Daniel Smith,daniel.smith@gmail.com,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,2020-03-12,1.381139,436.398423,69.043128
2,5307192536865304,86.98,2020-03-12 01:40:57+00:00,47178,William Doe,william.doe@gmail.com,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,2020-03-12,1.381139,120.131452,69.043128
3,5791957346866373,36.99,2020-03-24 09:52:40+00:00,47178,William Doe,william.doe@gmail.com,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,2020-03-24,1.449414,53.613837,53.613837
4,5235927587417736,36.99,2020-03-24 07:12:33+00:00,94720,Emile Tumson,emile.tumson@gmail.com,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,2020-03-24,1.449414,53.613837,53.613837


In [15]:
# round the decimal places
data = data.round({'order_price_CAD': 2, 'product_price_CAD': 2})
data.head()

,Order_id,total_price,created_at,customer_id,customer_name,email,id,product_id,product_sku,product_name,product_price,date,CAD_rate,order_price_CAD,product_price_CAD
0,4280852170163518,55.98,2020-03-10 11:59:46+00:00,23955,Daniel Smith,daniel.smith@gmail.com,317808,477365777,GOGPR-07,Glueberry OG Pre-Rolls,5.99,2020-03-10,1.367779,76.57,8.19
1,6431399815998774,315.97,2020-03-12 07:16:07+00:00,23955,Daniel Smith,daniel.smith@gmail.com,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,2020-03-12,1.381139,436.40,69.04
2,5307192536865304,86.98,2020-03-12 01:40:57+00:00,47178,William Doe,william.doe@gmail.com,997208,278266679,GPSTASH-01,Grandpa's Stash Dried Flower,49.99,2020-03-12,1.381139,120.13,69.04
3,5791957346866373,36.99,2020-03-24 09:52:40+00:00,47178,William Doe,william.doe@gmail.com,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,2020-03-24,1.449414,53.61,53.61
4,5235927587417736,36.99,2020-03-24 07:12:33+00:00,94720,Emile Tumson,emile.tumson@gmail.com,759658,573794972,SAGENSOUR-01,Sage N Sour Dried Flower,36.99,2020-03-24,1.449414,53.61,53.61


In [15]:
# a summary file with Order_id and the final price
data_Order = data[['Order_id','order_price_CAD']]

#### Two sets of data has been exported - one is the complete data with all the detailed information and one is the summary based only on Order_id and the final price(CAD) and saved the files in the output folder

In [16]:
data.to_csv('Output\data.csv',index = False)
data_Order.to_csv('Output\data_order.csv',index = False)

## Task 2: SQL

In this task, we would like you to persist the data obtained from task 1: upload your data to a SQL database of your choice (SQLite, MySQL, PostgreSQL ...).

Design the right data model for your data
 Create the SQL tables
 Import your data
The outcome of this task is:

SQL statements to create tables for the data
A Python script to upload the data (or the one from Task 1, extended)

In [17]:
# Get the columns of data file so that same structured table can be created in SQL
data.columns

Index(['Order_id', 'total_price', 'created_at', 'customer_id', 'customer_name',
       'email', 'id', 'product_id', 'product_sku', 'product_name',
       'product_price', 'date', 'CAD_rate', 'order_price_CAD',
       'product_price_CAD'],
      dtype='object')

In [18]:
# formatting the date column
data['date'] =data['date'].astype('datetime64[ns]')

#### Three tables order_table, customer_table, product_tabe created at SQL server and setting up connections of SQL and python and we are uploading the data in those tables

In [21]:
import pyodbc

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TOAVUDP\MSSQLSERVER01;'
                      'Database=AdventureWorks2012;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()
cursor.execute('SELECT * FROM AdventureWorks2012.dbo.order_table')
 
# Insert DataFrame to Table
for row in data.itertuples():
    cursor.execute('''
                INSERT INTO AdventureWorks2012.dbo.order_table (Order_id,total_price,created_at,date,CAD_rate,order_price_CAD)
                VALUES (?,?,?,?,?,?)
                ''',
                
                row.Order_id,
                row.total_price,
                row.created_at,
                row.date,
                row.CAD_rate,
                row.order_price_CAD
                        
                )
conn.commit()
    

In [22]:
import pyodbc

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TOAVUDP\MSSQLSERVER01;'
                      'Database=AdventureWorks2012;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()
cursor.execute('SELECT * FROM AdventureWorks2012.dbo.customer_table')
 
# Insert DataFrame to Table
for row in data.itertuples():
    cursor.execute('''
                INSERT INTO AdventureWorks2012.dbo.customer_table (Order_id,customer_id,customer_name,email)
                VALUES (?,?,?,?)
                ''',
                
                row.Order_id,
                row.customer_id,
                row.customer_name,
                row.email
                        
                )
conn.commit()
    

In [23]:
import pyodbc

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TOAVUDP\MSSQLSERVER01;'
                      'Database=AdventureWorks2012;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()
cursor.execute('SELECT * FROM AdventureWorks2012.dbo.product_table')
 
# Insert DataFrame to Table
for row in data.itertuples():
    cursor.execute('''
                INSERT INTO AdventureWorks2012.dbo.product_table (Order_id,id,product_id,product_sku,product_name,
                                                                           product_price,date,CAD_rate,product_price_CAD)
                VALUES (?,?,?,?,?,?,?,?,?)
                ''',
                
                row.Order_id,
                row.id,
                row.product_id,
                row.product_sku,
                row.product_name,
                row.product_price,
                row.date,
                row.CAD_rate,
                row.product_price_CAD
                        
                )
conn.commit()
    